<a href="https://colab.research.google.com/github/waveology/kitchen/blob/main/introducci%C3%B3n_a_netcdf_y_xarray" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introducción al NetCDF con xarray

Descarga de ficheros de datos

In [ ]:
## Datos de temperatura a 2 metros - ERA5 - [1987-2020]  -  res=0.25º
## -------------------------------------------------------------------
#!wget https://owncloud.aemet.es/index.php/s/Ns1t4FXmZoi5hK4/download -O era5_t2m_1987_2020_europa_025.rar
#!unrar e era5_t2m_1987_2020_europa_025.rar && era5_t2m_1987_2020_europa_025.nc

## Datos de temperatura a 2 metros - ERA5 - [1987-2020]  -  res=0.50º
## -------------------------------------------------------------------
#!wget https://owncloud.aemet.es/index.php/s/Bxjcb2T5pPq1PYP/download -O era5_t2m_1987_2020_europa_05.rar
#!unrar e era5_t2m_1987_2020_europa_05.rar && era5_t2m_1987_2020_europa_05.rar

Importamos las extensiones xarray y Matplotlib (gráficos)

In [13]:
import xarray as xr
import matplotlib.pyplot as plt

##Lectura de datos
---

In [27]:
ds = xr.open_dataset('/content/era5_t2m_1987_2020_europa_025.nc')

In [28]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 161, latitude: 161, time: 2484)
Coordinates:
  * longitude  (longitude) float32 -30.0 -29.75 -29.5 -29.25 ... 9.5 9.75 10.0
  * latitude   (latitude) float32 60.0 59.75 59.5 59.25 ... 20.5 20.25 20.0
  * time       (time) datetime64[ns] 1987-01-01T12:00:00 ... 2020-12-28T12:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-18 18:07:06 GMT by grib_to_netcdf-2.30.2: grib_to_n...

In [29]:
ds.data_vars

Data variables:
    t2m      (time, latitude, longitude) float32 ...

In [30]:
for variable in ds:
     print(variable)

t2m


In [31]:
ds['t2m']

<xarray.DataArray 't2m' (time: 2484, latitude: 161, longitude: 161)>
[64387764 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 -30.0 -29.75 -29.5 -29.25 ... 9.5 9.75 10.0
  * latitude   (latitude) float32 60.0 59.75 59.5 59.25 ... 20.5 20.25 20.0
  * time       (time) datetime64[ns] 1987-01-01T12:00:00 ... 2020-12-28T12:00:00
Attributes:
    units:      K
    long_name:  2 metre temperature

In [32]:
ds.t2m

<xarray.DataArray 't2m' (time: 2484, latitude: 161, longitude: 161)>
[64387764 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 -30.0 -29.75 -29.5 -29.25 ... 9.5 9.75 10.0
  * latitude   (latitude) float32 60.0 59.75 59.5 59.25 ... 20.5 20.25 20.0
  * time       (time) datetime64[ns] 1987-01-01T12:00:00 ... 2020-12-28T12:00:00
Attributes:
    units:      K
    long_name:  2 metre temperature

##Selección
---

xarray se construye sobre Numpy y por eso comparte muchos de sus métodos.

In [33]:
t2m = ds.t2m - 273.15

In [34]:
t2m.shape

(2484, 161, 161)

En Numpy seleccionaríamos elementos del array de esta manera:

In [36]:
t2m[0,:,:]

<xarray.DataArray 't2m' (latitude: 161, longitude: 161)>
array([[  5.2075195,   5.295746 ,   5.3334045, ..., -21.171967 ,
        -22.094345 , -22.632004 ],
       [  5.247528 ,   5.3569336,   5.4286804, ..., -21.794342 ,
        -22.155518 , -21.076675 ],
       [  5.322815 ,   5.4298706,   5.519287 , ..., -19.236618 ,
        -18.438965 , -17.120087 ],
       ...,
       [ 20.020813 ,  20.014954 ,  20.183167 , ...,  15.194824 ,
         15.619537 ,  16.172485 ],
       [ 20.08905  ,  20.153778 ,  20.34436  , ...,  15.28775  ,
         15.618347 ,  16.172485 ],
       [ 20.222015 ,  20.300842 ,  20.503174 , ...,  15.608948 ,
         15.904236 ,  16.246613 ]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -30.0 -29.75 -29.5 -29.25 ... 9.5 9.75 10.0
  * latitude   (latitude) float32 60.0 59.75 59.5 59.25 ... 20.5 20.25 20.0
    time       datetime64[ns] 1987-01-01T12:00:00

En xarray se usan los operadores ***isel*** y ***sel***
El primero trabaja sobre índices mientras que el segundo se basa en las coordenadas.


In [38]:
# Seleccionamos los datos correspondientes al primer valor de tiempo
# -------------------------------------------------------------------
t2m.isel(time=0)

<xarray.DataArray 't2m' (latitude: 161, longitude: 161)>
array([[ 7.614624 ,  7.6817017,  7.746399 , ..., -2.7726746, -3.8115234,
        -4.4221497],
       [ 7.7252197,  7.769928 ,  7.813446 , ..., -3.5279846, -4.3268433,
        -4.6703796],
       [ 7.8158264,  7.8605347,  7.8946533, ..., -3.4868164, -3.2303467,
        -2.8162231],
       ...,
       [20.889069 , 20.973785 , 21.034973 , ..., 29.998718 , 29.855194 ,
        29.419891 ],
       [20.953796 , 20.998505 , 21.060852 , ..., 30.106964 , 30.169312 ,
        29.734009 ],
       [20.958496 , 21.000854 , 21.026733 , ..., 30.161072 , 30.261078 ,
        30.145782 ]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -30.0 -29.75 -29.5 -29.25 ... 9.5 9.75 10.0
  * latitude   (latitude) float32 60.0 59.75 59.5 59.25 ... 20.5 20.25 20.0
    time       datetime64[ns] 1987-02-20T12:00:00

In [43]:
# Seleccionamos los datos correspondientes a un valor específico del tiempo
# --------------------------------------------------------------------------
t2m.sel(time='1987-02-20T12:00:00',method='nearest')

<xarray.DataArray 't2m' (latitude: 161, longitude: 161)>
array([[ 7.614624 ,  7.6817017,  7.746399 , ..., -2.7726746, -3.8115234,
        -4.4221497],
       [ 7.7252197,  7.769928 ,  7.813446 , ..., -3.5279846, -4.3268433,
        -4.6703796],
       [ 7.8158264,  7.8605347,  7.8946533, ..., -3.4868164, -3.2303467,
        -2.8162231],
       ...,
       [20.889069 , 20.973785 , 21.034973 , ..., 29.998718 , 29.855194 ,
        29.419891 ],
       [20.953796 , 20.998505 , 21.060852 , ..., 30.106964 , 30.169312 ,
        29.734009 ],
       [20.958496 , 21.000854 , 21.026733 , ..., 30.161072 , 30.261078 ,
        30.145782 ]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -30.0 -29.75 -29.5 -29.25 ... 9.5 9.75 10.0
  * latitude   (latitude) float32 60.0 59.75 59.5 59.25 ... 20.5 20.25 20.0
    time       datetime64[ns] 1987-02-20T12:00:00

In [ ]:
# Selección de un tiempo que no está presente en los datos
# ----------------------------------------------------------
t2m.sel(time='1987-02-18T12:00:00')
#t2m.sel(time='1987-02-18T12:00:00',method='nearest')

##Gráficos
---

##Estadística
---

##Agrupamientos
---

##Remuestreo
---

##Máscaras
---

##Múltiples fuentes
---

##Salida en NetCDF
---